In [1]:
from dotenv import find_dotenv,load_dotenv
from pprint import pprint
load_dotenv(find_dotenv(), override=True)

True

In [1]:
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import StructuredTool

In [6]:
class MultiplierInput(BaseModel):
    a: int = Field(description="First number")
    b: int = Field(description = "Second number")
    
def multiply(a: int, b:int)->int:
    return a*b

multiplier = StructuredTool.from_function(
    func=multiply,
    name="Multiplier",
    description="Multiply two numbers",
    args_schema=MultiplierInput,
    return_direct=False
)

In [9]:
multiplier.run({"a": 10,"b": 2})

20

In [27]:
class AdderInput(BaseModel):
    a: int = Field(description = "First number")
    b: int= Field(description= "Second number")
    
def addnum(a:int, b:int)->int:
    return a+b

adder = StructuredTool.from_function(
    func = addnum,
    name="addition",
    description="add two numbers",
    args_schema=AdderInput,
    return_direct=False
)
    

In [28]:
tools = [multiplier, adder]

In [29]:
adder.run({"a":10, "b":20})

30

In [30]:
from langgraph.prebuilt import ToolExecutor

tool_executor = ToolExecutor(tools)

In [31]:
from langchain_core.pydantic_v1 import BaseModel, Field

class Response(BaseModel):
    """Final answer to the user"""
    result:int = Field(description = "the result of the computation")
    explanation:str = Field(
        description = "Explanation of the steps taken to get the result"
    )

In [32]:
from langchain_openai import ChatOpenAI
from langchain.tools.render import format_tool_to_openai_function
from langchain_core.utils.function_calling import convert_pydantic_to_openai_function

llm = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0,streaming=True)

functions = [format_tool_to_openai_function(i) for i in tools]
functions.append(convert_pydantic_to_openai_function(Response))

model = llm.bind_functions(functions)

In [33]:
from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage


class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

In [34]:
from langgraph.prebuilt import ToolInvocation
import json
from langchain_core.messages import FunctionMessage

# Define the function that determines whether to continue or not
def should_continue(state):
    messages = state["messages"]
    last_message = messages[-1]
    # If there is no function call, then we finish
    if "function_call" not in last_message.additional_kwargs:
        return "end"
    elif last_message.additional_kwargs["function_call"]["name"] == "Response":
        return "end"
    # Otherwise if there is, we continue
    else:
        return "continue"


# Define the function that calls the model
def call_model(state):
    messages = state["messages"]
    response = model.invoke(messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}


# Define the function to execute tools
def call_tool(state):
    messages = state["messages"]
    # Based on the continue condition
    # we know the last message involves a function call
    last_message = messages[-1]
    # We construct an ToolInvocation from the function_call
    action = ToolInvocation(
        tool=last_message.additional_kwargs["function_call"]["name"],
        tool_input=json.loads(
            last_message.additional_kwargs["function_call"]["arguments"]
        ),
    )

    # response = input(prompt=f"[y/n] continue with: {action}?")
    # if response == "n":
    #     raise ValueError("User cancelled")
    
    # We call the tool_executor and get back a response
    response = tool_executor.invoke(action)
    # We use the response to create a FunctionMessage
    function_message = FunctionMessage(content=str(response), name=action.tool)
    # We return a list, because this will get added to the existing list
    return {"messages": [function_message]}

In [35]:
from langgraph.graph import StateGraph, END

# Initialize a new graph
graph = StateGraph(AgentState)

# Define the two "Nodes"" we will cycle between
graph.add_node("agent", call_model)
graph.add_node("action", call_tool)

# Define all our "Edges"
# Set the "Starting Edge" as "agent"
# This means that this node is the first one called
graph.set_entry_point("agent")

# We now add a "Conditional Edge"
# Conditinal agents take:
# - A start node
# - A function that determines which node to call next
# - A mapping of the output of the function to the next node to call
graph.add_conditional_edges(
    "agent",
    should_continue,
    {
        "continue": "action",
        # END is a special node marking that the graph should finish.
        "end": END,
    },
)

# We now add a "Normal Edge" that should always be called after another
graph.add_edge("action", "agent")

# We compile the entire workflow as a runnable
app = graph.compile()

In [36]:
from langchain_core.messages import HumanMessage

inputs = {
    "messages": [HumanMessage(content="what is the product of 37 and 49 plus 42?")]
}

for output in app.stream(inputs):
    
    for key, value in output.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value)
    print("\n---\n")

Output from node 'agent':
---
{'messages': [AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"a":37,"b":49}', 'name': 'Multiplier'}}, response_metadata={'finish_reason': 'function_call'}, id='run-28832d08-ac54-430d-8af5-2d583db8cf98-0')]}

---

Output from node 'action':
---
{'messages': [FunctionMessage(content='1813', name='Multiplier')]}

---

Output from node 'agent':
---
{'messages': [AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"a":1813,"b":42}', 'name': 'addition'}}, response_metadata={'finish_reason': 'function_call'}, id='run-bdc465fb-5363-4603-84ba-04a559571a40-0')]}

---

Output from node 'action':
---
{'messages': [FunctionMessage(content='1855', name='addition')]}

---

Output from node 'agent':
---
{'messages': [AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"result":1855,"explanation":"The product of 37 and 49 is 1813. Adding 42 to the product gives us a final result of 1855."}', 'name': 'Re

In [39]:
output = app.invoke({"messages":[HumanMessage(content="What is 100+2")]})

In [40]:
output

{'messages': [HumanMessage(content='What is 100+2'),
  AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"a":100,"b":2}', 'name': 'addition'}}, response_metadata={'finish_reason': 'function_call'}, id='run-34164fc4-5984-4ac8-97b8-8d55f642542b-0'),
  FunctionMessage(content='102', name='addition'),
  AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"result":102,"explanation":"The sum of 100 and 2 is 102."}', 'name': 'Response'}}, response_metadata={'finish_reason': 'function_call'}, id='run-2d3f59ad-108c-4c8b-b148-6601eb76d7d8-0')]}

In [53]:
app.get_graph().print_ascii()

                    +-----------+             
                    | __start__ |             
                    +-----------+             
                          *                   
                          *                   
                          *                   
                      +-------+               
                      | agent |               
                     *+-------+*              
                   **           ***           
                 **                **         
               **                    **       
+-----------------------+              **     
| agent_should_continue |               *     
+-----------------------+               *     
            *           *****           *     
            *                ****       *     
            *                    ***    *     
       +---------+                +--------+  
       | __end__ |                | action |  
       +---------+                +--------+  
